## Bibliography

* https://dev.to/louisbertin/download-all-flickr-photos-of-a-user-with-only-20-lines-of-python-4gh7
* https://medium.com/@jameld.pro/secure-your-api-keys-b123f30ac014

## Import flickr api keys and secrets


In [ ]:
import flickr_api
from config import api_key, api_secret

flickr_api.set_keys(api_key = api_key, api_secret = api_secret)


In [ ]:

from pathlib import Path

USER_NAME = '15K Nocturna Valencia Banco Mediolanum'

output_path = Path('flickr_images') / USER_NAME
output_path.mkdir(parents=True, exist_ok=True)


In [ ]:
try:
    user = flickr_api.Person.findByUserName(USER_NAME)
    print('User found: ' + user.username)
except Exception as e:
    print(e, 'User not found')


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import pandas as pd


def download_photo(photo):
    # Using private _getOutputFilename as a quick hack to retrieve file extension
    extension = photo._getOutputFilename('', None)
    
    # Get image metadata
    try:
        filename = f'{photo.title.replace("/", "-")}_{photo.id}{extension}'
        sizes = photo.getSizes()
        biggest_size = list(sizes.keys())[-1]
    except Exception as e:
        print(e, 'Error getting image metadata')
        
    # Get original image url
    try:
        url = sizes[biggest_size]['source']
    except Exception as e:
        print(e, 'Error getting image url')
    
    # Save image in local
    try:
        dest_path = output_path / filename
        if not dest_path.exists():
            photo.save(str(dest_path))
    except Exception as e:
        print(e, 'Error saving image ', filename)
        continue
    
    # Return metadata of the image
    new_item = {
        'filename': filename,
        'url': url,
    }
    return new_item


def get_all_photos(user):
    # Flickr API only allows you to retrieve photos on pages of up to 500 photos
    # This generator abstracts from that limitation so all photos are iterated
    photos = user.getPublicPhotos()
    page = 1
    while photos:
        yield from photos
        # Advance to the next page
        page += 1
        photos = user.getPublicPhotos(page=page)


csv_info_path = 'flickr_images/' + USER_NAME + '_info.csv',
images_info_df = pd.DataFrame(columns=['filename', 'url']),

total = user.getPublicPhotos().info.total

with ThreadPoolExecutor() as executor:
    for new_item in tqdm(executor.map(download_photo, get_all_photos(user)), total=total, unit='imgs'):
        # Append new item to info dataframe
        images_info_df = images_info_df.append(new_item, ignore_index=True)
        
# Saving csv info
images_info_df.to_csv(csv_info_path, index=False) 